# resp_evoked (quantifying response to evoked activity in a given session)
IMPORTANT: Keep this notebook as identical to `resp_photostim.ipynb` as possible.

Both scripts do 'trial-based' analysis, where each trial has some start/end time and identity.

The identity can be:
    1) evoked stimulation trial type (currently they are all the same)
    2) photostim stimulation trial type / the identity of the mark point being stimulated

The outputs for the 'resp_map-related' part are:
    1) 'response map'(s) of the movie field of view for each evoked trial (+summary over trials) - response of network to a particular trial type of evoked activity
    2) 'response map'(s) of the movie field of view for each photostim trial (+summary over trials) - response of network to a particular single cell stimulation

The outputs for the 'suite2p-related' part are:
    1) which cells are responding to a particular evoked stimulation
    2) which cells are responding to a particular photostim stimulation (the stimulated cell as well as all other cells)


In [ ]:
# channel: 2
# plane: 0
# frame_period: 0.033602476  # metadata-derived frame period for 30Hz acquisition
# fov_shape: [512, 512]      # shape of the FOV in pixels

# # baseline and response parameters
# bsln_n_frames: 10          # baseline window in frames
# resp_n_frames: 10          # response window in frames

# bsln_sub_type: "trial_by_trial"  # 'trial_by_trial' or 'session_wide'

# # spatial extent of response
# n_dist_bins: 724           # number of distance bins (724 = diagonal of 512x512)

# # visualization parameters
# n_rows_fov: 4
# vlim: 200
# txt_shift: [7, 7]
# sat_perc_fov: 99.99
# peristim_wind: [10, 30]
# zoomin_npix: 128

# dist_bins_xlim: 362        # 724 // 2
# dist_bins_xlim_zoom: 45    # 724 // 16

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from photostim_deve.response.io import get_all_tiff_paths, parse_evoked_protocol_csv
from photostim_deve.image_analysis.compute import get_resp_imgs
from photostim_deve.image_analysis.plot import plot_resp_imgs



In [ ]:
# params (based on resp_photostim_config.yaml)
channel = 2
plane = 0 
frame_period = 0.033602476 # metadata-derived frame period for 30Hz acquisition fov_shape = [512, 512] # shape of the FOV in pixels
fov_shape = [512, 512] # shape of the FOV in pixels

# skipping the rest of the parameters...

In [ ]:
bsln_dur = 500 # baseline duration in ms
resp_dur = 2000 # response duration in ms

frame_avg_mode = 'median'
trial_avg_mode = 'median'

plot_debug = True # whether to plot the baseline and response time periods and images for each trial for debugging purposes

In [ ]:
# set params
subject = 'jm064'
session = '2025-11-13_s'

In [ ]:
session_path = os.path.join('data_proc', 'jm', subject, session)

# tiff file paths
s2p_path = os.path.join(session_path, 'suite2p', f'plane{plane}')
tiff_dir = os.path.join(s2p_path, f'reg_tif_chan{channel}')
all_tiff_paths = get_all_tiff_paths(tiff_dir)

# stimulation protocol pathsa
csv_save_path = os.path.join('data_proc', 'jm', subject, session, 'evoked_protocol.csv')
csv_load_path = csv_save_path

# output paths
output_path = os.path.join(session_path, 'resp_evoked')
output_fig_path = os.path.join(output_path, 'fig')

if not os.path.exists(output_path):
    os.makedirs(output_path)
if not os.path.exists(output_fig_path):
    os.makedirs(output_fig_path)



In [ ]:
stim_times, stim_frames, stim_type = parse_evoked_protocol_csv(session_path, 
                                                               csv_load_path, 
                                                               frame_period=frame_period)

In [ ]:
resp_bsln, resp_resp, resp_diff, f_mean = get_resp_imgs(all_tiff_paths, 
                                                stim_frames, 
                                                stim_type, 
                                                frame_avg_mode=frame_avg_mode, 
                                                bsln_dur=bsln_dur, 
                                                resp_dur=resp_dur, 
                                                fov_shape=fov_shape, 
                                                frame_period=frame_period,
                                                plot_debug=plot_debug)

In [ ]:
stim_type = 0 # only one stim type in evoked protocol, so just set to 0 for plotting

if trial_avg_mode == 'mean':
    plot_resp_imgs(np.mean(resp_bsln[stim_type], axis=0), np.mean(resp_resp[stim_type], axis=0), np.mean(resp_diff[stim_type], axis=0), j=stim_type, l=0)
    plt.figure(figsize=(6, 6), dpi=300)
    plt.imshow(np.mean(resp_diff[stim_type], axis=0), vmin=-400, vmax=400, cmap='bwr')
    plt.axis('off')

elif trial_avg_mode == 'median':
    plot_resp_imgs(np.median(resp_bsln[stim_type], axis=0), np.median(resp_resp[stim_type], axis=0), np.median(resp_diff[stim_type], axis=0), j=stim_type, l=0)
    plt.figure(figsize=(6, 6), dpi=300)
    plt.imshow(np.median(resp_diff[stim_type], axis=0), vmin=-400, vmax=400, cmap='bwr')
    plt.axis('off')



In [ ]:
# Load track2p ROIs (cells tracked across all days from 'fake suite2p')
stat = np.load(os.path.join('data_proc/jm/jm064/track2p/matched_suite2p', session, 'suite2p', 'plane0', 'stat.npy'), allow_pickle=True)

In [ ]:
plt.figure(figsize=(6, 6), dpi=300)

if trial_avg_mode == 'mean':
    plt.imshow(np.mean(resp_diff[stim_type], axis=0), vmin=-400, vmax=400, cmap='bwr')
elif trial_avg_mode == 'median':
    plt.imshow(np.median(resp_diff[stim_type], axis=0), vmin=-400, vmax=400, cmap='bwr')

plt.axis('off')
for cell in stat:
    ypix = cell['ypix']
    xpix = cell['xpix']
    contour = np.zeros(fov_shape)
    contour[ypix, xpix] = 1
    plt.contour(contour, colors='k', linewidths=0.1, alpha=0.1)

In [ ]:
# get the intensity for each ROI and plot the distribution of intensities across all ROIs
intensities = []
for cell in stat:
    ypix = cell['ypix']
    xpix = cell['xpix']
    if trial_avg_mode == 'mean':
        intensity = np.mean(resp_diff[stim_type][:, ypix, xpix])
    elif trial_avg_mode == 'median':
        intensity = np.median(resp_diff[stim_type][:, ypix, xpix])
    intensities.append(intensity)

plt.figure(figsize=(8, 2), dpi=300)
plt.hist(intensities, bins=200)
plt.title(f'Distribution of mean intensity in ROIs (trial_avg_mode={trial_avg_mode})')
plt.xlabel('Mean intensity in ROI')

In [ ]:
foo # raise error to stop the code here for now

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# get five random numbers between 0 and 1312
np.random.seed(2)
rand_nrs = np.random.randint(0, 100, size=15)
print(rand_nrs)

In [ ]:
data_dir = 'data_proc' # data_loc is  the directory on local ssd (only two sessions, one for jm049 and one for jm048)
experimenter = 'jm'
mouse = 'jm064' # 'jm049' or 'jm048'
t2p_s2p_dir = 'track2p/matched_suite2p'

zscore_allF = True

stim_wind = (10, 120)

# plot_nrns = [7, 15, 226, 116, 67, 0, 1, 2, 3, 4, 684, 559, 1216, 835, 763] # jm055
# plot_nrns = [0, 1, 14, 35, 51, 4, 8, 22, 29, 2, 684, 559, 629, 192, 835] # jm056
# plot_nrns = [57, 47, 73, 92, 60, 87, 99, 21, 20, 79, 50, 16, 22, 82, 40] # jm064
plot_nrns = [57, 47, 73, 92, 60, 87, 99, 21, 20, 79, 50, 16, 22, 82, 40, 0, 1, 2, 150, 329]

# get 15 random numbers between 0 and 500
# plot_nrns = rand_nrs.tolist()
# labels = ['resp', 'resp', 'resp', 'resp', 'resp', 'state', 'state', 'state', 'state', 'state', 'rand', 'rand', 'rand', 'rand', 'rand']
labels = ['rand'] * 15

In [ ]:
all_F[0].shape

In [ ]:
import os

In [ ]:
def zscore_rows(F):
    F = np.asarray(F, dtype=float)
    mean = F.mean(axis=1, keepdims=True)
    std = F.std(axis=1, keepdims=True)
    return (F - mean) / std

In [ ]:
# for now go to the fake suitep2p directory and load all the Fs from theree
matched_s2p_path = os.path.join(data_dir, experimenter, mouse, t2p_s2p_dir)
# now get all the subdirectories and load F
all_matched_s2p_paths = [os.path.join(matched_s2p_path, d) for d in os.listdir(matched_s2p_path) if os.path.isdir(os.path.join(matched_s2p_path, d))]
all_matched_s2p_paths.sort()  # sort to have a consistent order

all_F = []
all_stim_protocol = []
all_stim_times = []

for path in all_matched_s2p_paths:

    print(f'Loading from {path}')
    F = np.load(os.path.join(path, 'suite2p', 'plane0', 'F.npy'))  # shape (n_neurons, n_frames)

    session = os.path.basename(path)
    stim_protocol = np.load(f'{data_dir}/{experimenter}/{mouse}/{session}/stim_protocol.npy')
    stim_times = np.load(f'{data_dir}/{experimenter}/{mouse}/{session}/stim_times.npy')

    if zscore_allF:
        F = zscore_rows(F)

    all_F.append(F)
    all_stim_protocol.append(stim_protocol)
    all_stim_times.append(stim_times)


In [ ]:
all_stim_times

In [ ]:
for (i,F) in enumerate(all_F):
    for j in plot_nrns:
        plt.figure(figsize=(20, 2))
        plt.plot(F[j], c=f'C{i}')
        for st in stim_times:
            plt.axvline(st, color='r')

In [ ]:
stim_array = np.zeros((len(all_F), len(plot_nrns), len(stim_times), stim_wind[0] + stim_wind[1]))

for k in range(len(all_F)):
    for i in range(len(plot_nrns)):
        for j, st in enumerate(all_stim_times[k]):
            stim_array[k, i, j, :] = all_F[k][plot_nrns[i], st - stim_wind[0]:st + stim_wind[1]]
            # subtract baseline
            stim_array[k, i, j, :] = stim_array[k, i, j, :] - np.mean(stim_array[k, i, j, :stim_wind[0]])
        

In [ ]:
resp_axlims = [-1.96, 1.96*2]
resp_axlims_st = [-1.96*4, 1.96*4] # single trial

In [ ]:
# now plot the heatmap for each neuron
for k in range(len(all_F)):
    this_stim_array = stim_array[k, :, :, :]  # average over stim times
    fig, axs = plt.subplots(3, 5, figsize=(10, 5), dpi=600)
    for i in range(len(plot_nrns)):
        ax = axs[i // 5, i % 5]
        plt.sca(ax)
        plt.imshow(this_stim_array[i, :, :], aspect='auto', cmap='bwr', vmin = resp_axlims_st[0], vmax = resp_axlims_st[1])
        # if first column, add ylabel
        # remove xticks and yticks
        plt.xticks([])
        plt.yticks([])
        if i % 5 == 0:
            plt.ylabel(labels[i])
            # add yticks
            plt.yticks([0, this_stim_array.shape[1]//2, this_stim_array.shape[1]-1], [0, this_stim_array.shape[1]//2, this_stim_array.shape[1]-1])
        # if last row, add xlabel
        if i // 5 == 2:
            plt.xlabel('Time (frames)')
            plt.xticks([stim_wind[0], stim_wind[0]+stim_wind[1]//2, stim_wind[0]+stim_wind[1]], [0, stim_wind[1]//2, stim_wind[1]])
                


In [ ]:
show_n_nrn = 20

for k in range(len(all_F)):
    this_stim_array = stim_array[k, :, :, :]  # average over stim times
    # plot the average +-std for resp neurons
    fig, axs = plt.subplots(1, show_n_nrn, figsize=(show_n_nrn * 2, 1.5), dpi=300)

    for i in range(show_n_nrn):
        ax = axs[i]
        plt.sca(ax)
        mean_trace = np.mean(this_stim_array[i, :, :], axis=0)
        std_trace = np.std(this_stim_array[i, :, :], axis=0)
        plt.plot(mean_trace, color=f'C{i}')
        plt.fill_between(np.arange(len(mean_trace)), mean_trace - std_trace, mean_trace + std_trace, color=f'C{i}', alpha=0.1)
        plt.title(f'Resp neuron {plot_nrns[i]}')
        plt.axvline(stim_wind[0], color='grey', linestyle='--')
        plt.xlabel('Time (frames)')
        if i == 0:
            plt.ylabel('dF/F')
        plt.xticks([stim_wind[0], stim_wind[0]+stim_wind[1]//2, stim_wind[0]+stim_wind[1]], [0, stim_wind[1]//2, stim_wind[1]])
        # keep y ticks but remove labels
        ax.set_yticklabels([])
        # remove top and right spines
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_ylim(resp_axlims)
